In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'evaluation'))

In [2]:
from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
from classification import compute_f1_score

In [3]:
def json_converter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

# Correct F1 score in Json For Synthetic Data

In [69]:
data_set = 'struct'
algorithm = 'microsoft_ssa'
results_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'test', algorithm, data_set)
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', data_set)
list_of_results = glob.glob(os.path.join(results_path, "*.json"))
for result in list_of_results:
    with open(result) as file:
        results_dict = json.load(file)
    ts_name = results_dict['data_name']
    detected = np.array(results_dict['cp'])
    labels = pd.read_csv(os.path.join(data_path, "{}_labels.csv".format(ts_name)), header=None)
    corrected_f1 = compute_f1_score(labels.values[:-1, :], detected, 10)
    print(ts_name + " old: " + str(results_dict['score']['value']), " new: " + str(corrected_f1))
    results_dict['score']['value'] = corrected_f1
    with open(result, 'w') as file:
        json.dump(results_dict, file)

struct10 old: 0.6666666666666666  new: 0.7499999999999999
struct17 old: 0.6666666666666666  new: 0.7272727272727272
struct13 old: 0.7272727272727272  new: 0.8000000000000002
struct19 old: 0.6666666666666666  new: 0.7499999999999999
struct9 old: 0.7692307692307692  new: 0.8333333333333333
struct18 old: 0.7272727272727272  new: 0.8000000000000002
struct4 old: 0.6666666666666666  new: 0.7272727272727272
struct12 old: 0.5454545454545454  new: 0.6
struct5 old: 0.5882352941176471  new: 0.625
struct1 old: 0.7692307692307692  new: 0.8333333333333333
struct7 old: 0.6666666666666666  new: 0.7272727272727272
struct0 old: 0.8333333333333334  new: 0.9090909090909091
struct8 old: 0.8333333333333334  new: 0.9090909090909091
struct2 old: 0.7692307692307692  new: 0.8333333333333333
struct11 old: 0.8  new: 0.888888888888889
struct14 old: 0.6153846153846153  new: 0.6666666666666666
struct15 old: 0.8  new: 0.888888888888889
struct6 old: 0.5454545454545454  new: 0.6
struct16 old: 0.6  new: 0.66666666666666

# Write table from jsons (REWRITES THE TABLE)

In [5]:
data_set = 'yahoo'
algorithm = 'mssa_mw'
results_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'test', algorithm, data_set)
list_of_results = glob.glob(os.path.join(results_path, "*.json"))
table = []
for result in list_of_results:
    with open(result, 'r') as file:
        results_dict = json.load(file)
    ts_name = results_dict['data_name']
    status = results_dict['status']
    score = results_dict['score']['value']
    row = [algorithm, data_set, ts_name, status, score]
    table.append(row)
table = pd.DataFrame(table)
table.to_csv('{}/results_table.csv'.format(results_path), index=False, header=False)
